# Out-of-State-Contributions: Candidates Analysis

How much out-of-state money are candidates raising in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [1]:
from functools import reduce
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

Import contributions data.

In [2]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/joe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,14,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362352 entries, 0 to 6362351
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(1), int64(2), object(16)
memory usage: 922.3+ MB


<IPython.core.display.Javascript object>

Convert the contribution date and latest month columns to datetime data type.

In [4]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362352 entries, 0 to 6362351
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(2), object(14)
memory usage: 922.3+ MB


## Calculate out-of-state contributions by candidate

Filter the data by election cycle.

In [5]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_14 = contributions[contributions["year"] == 2014]
contributions_10 = contributions[contributions["year"] == 2010]

Group by candidate, state, year, office status, redistricting role (for 2018 cycle data) and in-vs.-out-of-state contribution status and sum contributions.

In [6]:
contributions_by_candidate_18 = contributions_18.groupby(["candidate", "state", "year", "standardized_office", "standardized_status", "redistricting_role", "in_out_state"])["amount"].sum().reset_index()
contributions_by_candidate_18.rename(columns={"amount": "amount_18"}, inplace=True)
contributions_by_candidate_14 = contributions_14.groupby(["candidate", "state", "year", "standardized_office", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_candidate_14.rename(columns={"amount": "amount_14"}, inplace=True)
contributions_by_candidate_10 = contributions_10.groupby(["candidate", "state", "year", "standardized_office", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_candidate_10.rename(columns={"amount": "amount_10"}, inplace=True)

Pivot the dataframes to aggregate each candidate's data in a single row.

In [7]:
contributions_by_candidate_18 = pd.pivot_table(contributions_by_candidate_18, index=["candidate", "state", "year", "standardized_office", "standardized_status", "redistricting_role"], columns=["in_out_state"]).reset_index()
contributions_by_candidate_14 = pd.pivot_table(contributions_by_candidate_14, index=["candidate", "state", "year", "standardized_office", "standardized_status"], columns=["in_out_state"]).reset_index()
contributions_by_candidate_10 = pd.pivot_table(contributions_by_candidate_10, index=["candidate", "state", "year", "standardized_office", "standardized_status"], columns=["in_out_state"]).reset_index()

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [8]:
contributions_by_candidate_18.fillna(0, inplace=True)
contributions_by_candidate_14.fillna(0, inplace=True)
contributions_by_candidate_10.fillna(0, inplace=True)

Flatten the resulting dataframes' multi-index columns.

In [9]:
contributions_by_candidate_18.columns = ["_".join(column).replace("-","_").strip("_") for column in contributions_by_candidate_18.columns.values]
contributions_by_candidate_18.rename(columns={"standardized_office": "standardized_office_18"}, inplace=True)
contributions_by_candidate_14.columns = ["_".join(column).replace("-","_").strip("_") for column in contributions_by_candidate_14.columns.values]
contributions_by_candidate_14.rename(columns={"standardized_office": "standardized_office_14"}, inplace=True)
contributions_by_candidate_10.columns = ["_".join(column).replace("-","_").strip("_") for column in contributions_by_candidate_10.columns.values]
contributions_by_candidate_10.rename(columns={"standardized_office": "standardized_office_10"}, inplace=True)

Calculate the proportion of in-state, out-of-state and unknown contributions.

In [10]:
contributions_by_candidate_18["pct_18_in_state"] = contributions_by_candidate_18["amount_18_in_state"] / (contributions_by_candidate_18["amount_18_in_state"] + contributions_by_candidate_18["amount_18_out_of_state"] + contributions_by_candidate_18["amount_18_unknown"])
contributions_by_candidate_18["pct_18_out_of_state"] = contributions_by_candidate_18["amount_18_out_of_state"] / (contributions_by_candidate_18["amount_18_in_state"] + contributions_by_candidate_18["amount_18_out_of_state"] + contributions_by_candidate_18["amount_18_unknown"])
contributions_by_candidate_18["pct_18_unknown"] = contributions_by_candidate_18["amount_18_unknown"] / (contributions_by_candidate_18["amount_18_in_state"] + contributions_by_candidate_18["amount_18_out_of_state"] + contributions_by_candidate_18["amount_18_unknown"])
contributions_by_candidate_14["pct_14_in_state"] = contributions_by_candidate_14["amount_14_in_state"] / (contributions_by_candidate_14["amount_14_in_state"] + contributions_by_candidate_14["amount_14_out_of_state"] + contributions_by_candidate_14["amount_14_unknown"])
contributions_by_candidate_14["pct_14_out_of_state"] = contributions_by_candidate_14["amount_14_out_of_state"] / (contributions_by_candidate_14["amount_14_in_state"] + contributions_by_candidate_14["amount_14_out_of_state"] + contributions_by_candidate_14["amount_14_unknown"])
contributions_by_candidate_14["pct_14_unknown"] = contributions_by_candidate_14["amount_14_unknown"] / (contributions_by_candidate_14["amount_14_in_state"] + contributions_by_candidate_14["amount_14_out_of_state"] + contributions_by_candidate_14["amount_14_unknown"])
contributions_by_candidate_10["pct_10_in_state"] = contributions_by_candidate_10["amount_10_in_state"] / (contributions_by_candidate_10["amount_10_in_state"] + contributions_by_candidate_10["amount_10_out_of_state"] + contributions_by_candidate_10["amount_10_unknown"])
contributions_by_candidate_10["pct_10_out_of_state"] = contributions_by_candidate_10["amount_10_out_of_state"] / (contributions_by_candidate_10["amount_10_in_state"] + contributions_by_candidate_10["amount_10_out_of_state"] + contributions_by_candidate_10["amount_10_unknown"])
contributions_by_candidate_10["pct_10_unknown"] = contributions_by_candidate_10["amount_10_unknown"] / (contributions_by_candidate_10["amount_10_in_state"] + contributions_by_candidate_10["amount_10_out_of_state"] + contributions_by_candidate_10["amount_10_unknown"])

Join the 2018, 2014 and 2010 contributions by candidate data

In [12]:
list_of_contributions_by_candidate = [contributions_by_candidate_18, contributions_by_candidate_14, contributions_by_candidate_10]
contributions_by_candidate = reduce(lambda left, right: pd.merge(left, right, on=["candidate", "state"], how="outer"), list_of_contributions_by_candidate)
contributions_by_candidate.drop(["year_x", "year_y", "year"], axis=1, inplace=True)
contributions_by_candidate.rename(columns={"standardized_status_x": "standardized_status_18", "standardized_status_y": "standardized_status_14"}, inplace=True)
contributions_by_candidate.head()

,candidate,state,standardized_office_18,18_standardized_status,redistricting_role,amount_18_in_state,amount_18_out_of_state,amount_18_unknown,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,standardized_office_14,14_standardized_status,amount_14_in_state,amount_14_out_of_state,amount_14_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,standardized_office_10,standardized_status,amount_10_in_state,amount_10_out_of_state,amount_10_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,"ABBATE, PETER",NY,STATE HOUSE/ASSEMBLY,ADVANCED TO GENERAL,Y,"183,575.00","18,350.00","31,925.00",0.79,0.08,0.14,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan
1,"ABBOTT, DAVID H",IN,STATE HOUSE/ASSEMBLY,ADVANCED TO GENERAL,Y,"26,065.00",0.00,0.00,1.00,0.00,0.00,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan
2,"ABBOTT, GREG",TX,GOVERNOR/LIEUTENANT GOVERNOR,ADVANCED TO GENERAL,Y,"59,614,107.87","4,528,077.58","1,020.00",0.93,0.07,0.00,GOVERNOR/LIEUTENANT GOVERNOR,ADVANCED TO GENERAL,"22,988,410.36","1,488,413.64","4,791,017.12",0.79,0.05,0.16,NaN,NaN,nan,nan,nan,nan,nan,nan
3,"ABDUL-RAHIM, ANEES",MD,STATE HOUSE/ASSEMBLY,DID NOT ADVANCE,Y,"8,841.07",0.00,0.00,1.00,0.00,0.00,NaN,NaN,nan,nan,nan,nan,nan,nan,NaN,NaN,nan,nan,nan,nan,nan,nan
4,"ABERCROMBIE, CATHERINE F",CT,STATE HOUSE/ASSEMBLY,ADVANCED TO GENERAL,N,"2,485.00",0.00,0.00,1.00,0.00,0.00,STATE HOUSE/ASSEMBLY,ADVANCED TO GENERAL,"5,130.00",7.00,20.00,0.99,0.00,0.00,NaN,NaN,nan,nan,nan,nan,nan,nan


Export the data to Excel.

In [11]:
%%notify
contributions_by_candidate.to_excel("data/contributions_by_candidate.xlsx", index=False)

<IPython.core.display.Javascript object>